In [1]:
import pandas as pd
import lightgbm as lgb
import sklearn
import gplearn
import numpy as np
import gc
import os

In [2]:
from gplearn.genetic import SymbolicTransformer

PATH = '/home/kai/data/kaggle/homecredit/'

train = pd.read_pickle(PATH + 'train_total.pkl')
test = pd.read_pickle(PATH + 'test_total.pkl')
print('done')

done


# Get cols

In [3]:
target_array = train['TARGET'].values

train_gp = train[['EXT_SOURCE_2','EXT_SOURCE_3']]
test_gp = test[['EXT_SOURCE_2','EXT_SOURCE_3']]

train_gp = train_gp.dropna(axis = 0)
train_index = train_gp.index
test_gp = test_gp.dropna(axis = 0)

train_array = train_gp.values
test_array = test_gp.values
target_array = train['TARGET'].iloc[train_index]

In [4]:
len(target_array)
target_array

0         1
2         0
5         0
6         0
7         0
8         0
10        0
11        0
12        0
13        0
15        0
16        0
18        0
19        0
21        0
22        0
23        0
24        0
25        0
26        1
27        0
28        0
30        0
32        0
33        0
35        0
36        0
37        0
38        0
39        0
         ..
307476    0
307477    0
307478    0
307479    0
307480    0
307481    1
307482    0
307483    0
307485    0
307486    0
307487    0
307488    0
307489    1
307490    0
307491    0
307492    0
307493    0
307494    0
307495    0
307496    0
307497    0
307498    0
307499    0
307500    0
307502    0
307503    0
307505    0
307508    0
307509    1
307510    0
Name: TARGET, Length: 246116, dtype: int64

In [5]:
# function_set = ['add','sub', 'mul','div','sqrt', 'log', 'abs', 'neg', 'inv', 'max', 'min']


function_set = ['add','sub', 'mul','div', 'abs', 'neg', 'max', 'min', 'sqrt', 'sin', 'cos', 'log', 'inv']
gp = SymbolicTransformer(generations=5,
                         population_size=5000, 
                         stopping_criteria = 1, 
                         tournament_size = 500,
                         metric = 'spearman',
                         hall_of_fame=100, 
                         n_components=15,
                         function_set=function_set,
                         parsimony_coefficient=0.0001,
                         max_samples=1, 
                         verbose=1,
                         random_state=18,
                         n_jobs=30,
                        p_subtree_mutation=0.03,
                        p_hoist_mutation = 0.04,
                        p_point_mutation = 0.03,
                         const_range = (-10,10),
                         init_depth = (3,4),
                         p_point_replace = 0.2
                        )

# SymbolicTransformer(const_range=(-1.0, 1.0),
#           function_set=['add', 'sub', 'mul', 'div', 'abs', 'neg', 'max', 'min', 'sqrt', 'sin', 'cos', 'log', 'inv'],
#           generations=25, hall_of_fame=100, init_depth=(2, 6),
#           init_method='half and half', max_samples=0.9, metric='spearman',
#           n_components=10, n_jobs=20, p_crossover=0.9,
#           p_hoist_mutation=0.01, p_point_mutation=0.01,
#           p_point_replace=0.05, p_subtree_mutation=0.01,
#           parsimony_coefficient=0.0005, population_size=500,
#           random_state=0, stopping_criteria=0.95, tournament_size=20,
#           verbose=1, warm_start=False)

In [6]:
# %env JOBLIB_TEMP_FOLDER=/tmp
# %env JOBLIB_TEMP_FOLDER=/home/kai/disksata/tmp
gp.fit(train_array, target_array)

    |    Population Average   |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     9.53   0.111478636842       10   0.206810056153              N/A      2.14m
   1     7.16   0.165214676722       14   0.209179824566              N/A      1.61m
   2     8.64   0.137923289189       12   0.209691181072              N/A      1.14m
   3    12.37   0.125688369023       16   0.209691181072              N/A     43.16s
   4     9.33   0.119846203351        9   0.210366042958              N/A      0.00s


SymbolicTransformer(const_range=(-10, 10),
          function_set=['add', 'sub', 'mul', 'div', 'abs', 'neg', 'max', 'min', 'sqrt', 'sin', 'cos', 'log', 'inv'],
          generations=5, hall_of_fame=100, init_depth=(3, 4),
          init_method='half and half', max_samples=1, metric='spearman',
          n_components=15, n_jobs=30, p_crossover=0.9,
          p_hoist_mutation=0.04, p_point_mutation=0.03,
          p_point_replace=0.2, p_subtree_mutation=0.03,
          parsimony_coefficient=0.0001, population_size=5000,
          random_state=18, stopping_criteria=1, tournament_size=500,
          verbose=1, warm_start=False)

In [7]:
gp_features_train = gp.transform(train_array)
print('train done')

train done


In [8]:
df = pd.DataFrame(data = gp_features_train, index = train_index)
x = df[[0]]
df = pd.concat([train, x], axis = 1)

In [9]:
df.to_pickle(PATH + 'train_gp.pkl')

In [11]:
gp_features_test = gp.transform(test_array)
df = pd.DataFrame(data = gp_features_test)
x = df[[0]]
df = pd.concat([test, x], axis = 1)
df.to_pickle(PATH + 'test_gp.pkl')
df

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,NAME_CONTRACT_STATUS_Canceled_y,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA,POS_CASH_balance_MONTHS_BALANCE_sum,POS_CASH_balance_MONTHS_BALANCE_min,POS_CASH_balance_MONTHS_BALANCE_max,0
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0.0,2.0,0.0,0.0,0.0,0.0,-653.0,-96.0,-53.0,0.997805
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0.0,1.0,0.0,0.0,1.0,0.0,-220.0,-25.0,-15.0,1.031792
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0.0,3.0,0.0,0.0,1.0,0.0,-1064.0,-66.0,-3.0,0.983191
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0.0,2.0,0.0,0.0,0.0,0.0,-1301.0,-59.0,-20.0,0.983191
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0.0,1.0,0.0,0.0,0.0,0.0,-273.0,-27.0,-15.0,0.983191
5,100042,Cash loans,F,Y,Y,0,270000.0,959688.0,34600.5,810000.0,...,0.0,6.0,0.0,0.0,0.0,0.0,-3602.0,-90.0,-21.0,0.983191
6,100057,Cash loans,M,Y,Y,2,180000.0,499221.0,22117.5,373500.0,...,0.0,2.0,0.0,0.0,1.0,0.0,-259.0,-39.0,-2.0,0.983191
7,100065,Cash loans,M,N,Y,0,166500.0,180000.0,14220.0,180000.0,...,0.0,1.0,0.0,0.0,0.0,0.0,-219.0,-39.0,-34.0,0.983191
8,100066,Cash loans,F,N,Y,0,315000.0,364896.0,28957.5,315000.0,...,0.0,3.0,0.0,0.0,1.0,0.0,-653.0,-27.0,-1.0,1.176393
9,100067,Cash loans,F,Y,Y,1,162000.0,45000.0,5337.0,45000.0,...,0.0,4.0,0.0,0.0,0.0,0.0,-2312.0,-93.0,-1.0,0.983191


In [15]:
print(len(x),len(x[x[0]!= np.nan]))
len(df)

40075 40075


48744

In [18]:
train_new = pd.DataFrame(data = gp_features_train, index = train_index)
train = pd.concat([train_original, train_new], axis = 1)
print('train done')
val_original = pd.read_feather(Path + 'val_cleaned.ftr')
val_index = val.original.index
gp_features_val = gp_features_val = gp.transform(val_array)
val_new = pd.DataFrame(data = gp_features_val, index = val_index)
val = pd.concat([val_original, val_new], axis = 1)

train done


In [19]:
print(train.shape, val.shape)

(794952, 128) (22996974, 128)


In [20]:
val.to_pickle(Path + 'val_gp1.pkl')

In [21]:
train.to_pickle(Path + 'train_gp1.pkl')

In [23]:
test_supplement = pd.read_feather(Path + 'test_cleaned.ftr')
with open('./log.txt', 'a') as f: f.write('test rading done\n')
test_index = np.load('/home/kai/data/kaggle/talkingdata/data/supplement2testInexFinal.npy')
test = test_supplement.iloc[test_index]
print(test.shape)

(18790469, 118)


In [24]:
test_index = test.index

In [27]:
# print(test_index)
test_array = test[cols].values

In [28]:
gp_features_test = gp.transform(test_array)
test_new = pd.DataFrame(data = gp_features_test, index = test_index)
test = pd.concat([test, test_new], axis = 1)

In [29]:
test.to_pickle(Path + 'test_gp1.pkl')